In [ ]:
from flask import Flask, request, jsonify, render_template
import tensorflow as tf
import numpy as np
import cv2
from sklearn.pipeline import Pipeline
import os
from modules.preprocess import Resize, GaussianBlur, CLAHE_Color, Normalize, show_images
from modules.heatmap import saliency_map, visualize_saliency
import base64
from io import BytesIO
from tensorflow import keras

model = keras.models.load_model('skin_cancer.keras')
preprocess_pipeline = Pipeline([
        ('resize', Resize((128, 128))),
        ('blur', GaussianBlur()),
        ('clahe', CLAHE_Color())
    ])

def preprocess_and_load_image(image_path):
    try:
        img = cv2.imread(image_path)
        if img is None:
            return None
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        processed_img = preprocess_pipeline.transform([img_rgb])[0]
        return img_rgb, processed_img
    except Exception as e:
        print(f"Erro ao pré-processar/carregar imagem: {e}")
        return None, None

In [19]:
file_path = f'../data/03_primary/ISIC_2024/images/ISIC_0087605.jpg' #'ISIC_0052003.jpg'
original_image, processed_image = preprocess_and_load_image(file_path)



In [22]:
original_image.shape

(125, 125, 3)

In [23]:
processed_image.shape

(128, 128, 3)

In [11]:
model

<Sequential name=sequential_19, built=True>

In [24]:
file_path = f'../data/03_primary/ISIC_2024/images/ISIC_0087605.jpg' #'ISIC_0052003.jpg'
original_image, processed_image = preprocess_and_load_image(file_path)

prediction = model.predict(np.expand_dims(processed_image, axis=0))
probability = prediction[0][0]
class_label = "maligno" if probability > 0.5 else "benigno"

# Gerar o mapa de saliência
saliency = saliency_map(model, processed_image)

# Visualizar o mapa de saliência sobre a imagem original
saliency_overlayed = visualize_saliency(original_image, saliency)

# Codificar a imagem com o mapa de saliência para base64
_, img_encoded = cv2.imencode('.jpg', saliency_overlayed)
base64_image = base64.b64encode(img_encoded).decode('utf-8')

c:\Users\samir\OneDrive\Área de Trabalho\projeto_pdi\skin_cancer_detection\.conda\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step


In [28]:
def predict():
    try:
        file_path = f'../data/03_primary/ISIC_2024/images/ISIC_0087605.jpg' #'ISIC_0052003.jpg'
        original_image, processed_image = preprocess_and_load_image(file_path)
        
        prediction = model.predict(np.expand_dims(processed_image, axis=0))
        probability = prediction[0][0]
        class_label = "maligno" if probability > 0.5 else "benigno"

        # Gerar o mapa de saliência
        saliency = saliency_map(model, processed_image)

        # Visualizar o mapa de saliência sobre a imagem original
        saliency_overlayed = visualize_saliency(original_image, saliency)

        # Codificar a imagem com o mapa de saliência para base64
        _, img_encoded = cv2.imencode('.jpg', saliency_overlayed)
        base64_image = base64.b64encode(img_encoded).decode('utf-8')

        return {'probability': float(probability), 'class': class_label, 'saliency_image':base64_image}
    except Exception as e:
        print(f"Erro na previsão: {e}")
        return None

In [29]:
resp = predict()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


In [30]:
resp

{'probability': 0.5086302757263184,
 'class': 'maligno',
 'saliency_image': '/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/2wBDAQICAgICAgUDAwUKBwYHCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgr/wAARCAB9AH0DASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD5YeGNNv8AC1D7Eg3j5VSnTJ8zbJPm/geo5oZPK2fwt8z1/VB8uOheR9ok/i+VEqSbY8v2tE+5Q6QJt2VHs2QfJ8296AJ